In [5]:
import pandas as pd
from glob import glob
import os
import numpy as np

In [14]:
file = 'C:\\Users\\jlandman\\Desktop\\newData\\RU_Kutuzov\\GlaThiDa_2-0_Kutuzov_Johannes.xls'
print(file)

C:\Users\jlandman\Desktop\newData\RU_Kutuzov\GlaThiDa_2-0_Kutuzov_Johannes.xls


In [15]:
thin_dict = {(2072, 'DJANKUAT'): 2,
             (2073, 'DJANKUAT'): 2,
             (2074, 'MARUKH'): 2,
}

In [20]:
data_new = pd.DataFrame([])

data = pd.read_excel(file, sheetname='TTT - GL. THICKNESS POINT DATA')


for gid in np.unique(data.GlaThiDa_ID.values):
    temp = data[data.GlaThiDa_ID==gid]

    gname = temp[temp.GlaThiDa_ID == gid].GLACIER_NAME.values[0]
    pol_u = temp[temp.GlaThiDa_ID == gid].POLITICAL_UNIT.values[0]
    surv_date = temp[temp.GlaThiDa_ID == gid].SURVEY_DATE.values[0]
    remarks = temp[temp.GlaThiDa_ID == gid].REMARKS.iloc[0]
    gtd_id = gid

    temp = temp.groupby(temp.index // thin_dict[(gid, gname)]).mean() # average over every x values as specified above

    temp['POLITICAL_UNIT'] = pol_u
    temp['GLACIER_NAME'] = gname
    temp['REMARKS'] = remarks
    temp['DATA_FLAG'] = np.nan
    temp['GlaThiDa_ID'] = gtd_id
    temp['SURVEY_DATE'] = surv_date
    temp['POINT_ID'] = range(1,len(temp)+1)


    data_new = data_new.append(temp, ignore_index=True)

    
data_new['THICKNESS'] = data_new['THICKNESS'].map('{:.1f}'.format)
data_new = data_new[['GlaThiDa_ID','POLITICAL_UNIT', 'GLACIER_NAME', 'SURVEY_DATE', 'POINT_ID',  'POINT_LAT',  'POINT_LON',  'ELEVATION', 
                     'THICKNESS', 'THICKNESS_UNCERTAINTY', 'DATA_FLAG', 'REMARKS']]

data_new = data_new.sort_values(by=['GlaThiDa_ID','POINT_ID'])
data_new.to_excel(os.path.join(os.path.dirname(file),file.split('.')[0]+'_thinned.xlsx'), index=False)

   GlaThiDa_ID POLITICAL_UNIT GLACIER_NAME  SURVEY_DATE  POINT_ID  POINT_LAT  \
0         2072             SU     DJANKUAT     20120710         1  43.203950   
1         2072             SU     DJANKUAT     20120710         2  43.203959   
2         2072             SU     DJANKUAT     20120710         3  43.203963   
3         2072             SU     DJANKUAT     20120710         4  43.203961   
4         2072             SU     DJANKUAT     20120710         5  43.203954   

   POINT_LON  ELEVATION  THICKNESS  THICKNESS_UNCERTAINTY  DATA_FLAG  \
0  42.746723     2726.0      11.71                 0.3513        NaN   
1  42.746751     2726.0      11.21                 0.3363        NaN   
2  42.746786     2726.0      10.82                 0.3246        NaN   
3  42.746822     2726.0      10.49                 0.3147        NaN   
4  42.746867     2726.0      10.30                 0.3090        NaN   

                                             REMARKS  
0  Survey Method: GPRt VIRL6. A